In [1]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import os
import datetime as dt
import nltk
import string
import matplotlib.pyplot as plt
import pickle
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import product

### reading required files

In [5]:
sponsors = pd.read_csv("../Data/Sponsors.csv",index_col=0)
ids_by_year = pickle.load(open("../Data/ids_by_year_fda_reg_with_pos.pkl", "rb"))
sponsors.shape

(450665, 5)

In [82]:
ids_by_year_rev = {year:{val:key for key,val in ids_by_year[year].items()} for year in ids_by_year}

In [24]:
sponser_new_names = pd.read_csv('../Data/sponsor2new_name.csv')
sponser_new_names.shape

(4401, 4)

In [38]:
sponsor_old_2_new = sponser_new_names.set_index('Previous name')['Current name'].to_dict()
len(sponsor_old_2_new)

4401

In [27]:
nct_id_2_sponsor = pickle.load(open("../Data/nct_id_2_sponsor.pkl","rb"))
len(nct_id_2_sponsor)

11484

### working with primary sponsors

In [37]:
len(set(nct_id_2_sponsor_wo_date.values()).intersection(set(sponser_new_names['Previous name'].tolist())))

2779

In [30]:
nct_id_2_sponsor_wo_date = {key:val[0] for key,val in nct_id_2_sponsor.items()}

In [39]:
nct_id_2_sponsor_wo_date = {key:sponsor_old_2_new[val] for key,val in nct_id_2_sponsor_wo_date.items()}

In [48]:
unique_sponsors = sorted(list(set(nct_id_2_sponsor_wo_date.values())))
len(unique_sponsors)

2753

### sponsor_2_nct_ids by year

In [62]:
year_list = list(range(2014,2019))
year_list

[2014, 2015, 2016, 2017, 2018]

In [64]:
nct_ids_by_year = {year:list(ids_by_year[year].keys()) for year in year_list}

In [65]:
[(key,len(val)) for key,val in nct_ids_by_year.items()]

[(2014, 709), (2015, 963), (2016, 1643), (2017, 4750), (2018, 3419)]

In [67]:
nct_id_2_sponsor_by_year = {year:{nct_id:nct_id_2_sponsor_wo_date[nct_id] for nct_id in nct_ids_by_year[year]} for year in year_list}


In [70]:
%%time
sponsor_2_nct_ids_by_year = {year:{spons:[key for key,val in nct_id_2_sponsor_by_year[year].items() if val==spons] for spons in nct_id_2_sponsor_by_year[year].values()} for year in year_list}


CPU times: user 2.09 s, sys: 7.92 ms, total: 2.1 s
Wall time: 2.11 s


In [71]:
[(key,len(val)) for key,val in sponsor_2_nct_ids_by_year.items()]

[(2014, 245), (2015, 327), (2016, 540), (2017, 1644), (2018, 1329)]

In [78]:
sponsor_by_year_with_pos = {year:{elem:i for i,elem in enumerate(sorted(list(sponsor_2_nct_ids_by_year[year].keys())))} for year in year_list}
sponsor_by_year_with_pos_rev = {year:{i:elem for i,elem in enumerate(sorted(list(sponsor_2_nct_ids_by_year[year].keys())))} for year in year_list}


In [115]:
# pickle.dump(sponsor_2_nct_ids_by_year,
#             open("../Data/sponsor_2_nct_ids_by_year.pkl","wb"),
#             protocol=2)

# pickle.dump(sponsor_by_year_with_pos,
#             open("../Data/sponsor_by_year_with_pos.pkl","wb"),
#             protocol=2)

### loading final text model

In [6]:
# final_text_models = pickle.load(open("../Output/final_text_model.pkl",
#                                      "rb"))

In [10]:
for year in final_text_models:
    final_text_models[year] = final_text_models[year].todense()

### company level similarity matrix generation

In [106]:
company_sim_mat_by_year = {year:np.zeros((len(sponsor_by_year_with_pos_rev[year]),
                                         len(sponsor_by_year_with_pos_rev[year]))) for year in year_list}

[(year,company_sim_mat_by_year[year].shape) for year in year_list]

[(2014, (245, 245)),
 (2015, (327, 327)),
 (2016, (540, 540)),
 (2017, (1644, 1644)),
 (2018, (1329, 1329))]

In [107]:
#setting diagonal to 1
for year in year_list:
    for i in range(company_sim_mat_by_year[year].shape[0]):
        company_sim_mat_by_year[year][i,i]=1

In [111]:
%%time

#filling in non-diagonal elements

for year in year_list:
    for i in range(len(sponsor_by_year_with_pos_rev[year])):
        for j in range(i):
            spons_a, spons_b = sponsor_by_year_with_pos_rev[year][i], sponsor_by_year_with_pos_rev[year][j]
            nct_id_tup_list = [elem for elem in product(sponsor_2_nct_ids_by_year[year][spons_a], 
                                                        sponsor_2_nct_ids_by_year[year][spons_b])]
            avg_trial_sim = np.mean([final_text_models[year][ids_by_year[year][elem[0]],
                                                             ids_by_year[year][elem[1]]] for elem in nct_id_tup_list])
            company_sim_mat_by_year[year][i,j] = avg_trial_sim
            company_sim_mat_by_year[year][j,i] = avg_trial_sim

CPU times: user 1min 1s, sys: 605 ms, total: 1min 2s
Wall time: 1min 2s
Compiler : 187 ms


In [114]:
# pickle.dump(company_sim_mat_by_year,
#             open("../Output/final_text_company_similarity_model.pkl","wb"),
#             protocol=2)